In [1]:
import os
import sys
%pylab

import numpy as np
import pandas as pd
import ipywidgets
from scipy.optimize import Bounds
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    %pylab
from src import data

from src import efficient_frontier
from src import filter_assets
from src import plot
from src import portfolio
import numpy as np
import pandas as pd
import ipywidgets
from scipy.optimize import Bounds


Using matplotlib backend: <object object at 0x0000023A21B829D0>
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib
Using matplotlib backend: TkAgg
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
weights = np.array([1/3,1/3,1/3])
min_esg_score = 1300
max_esg_score = 2000
df = pd.read_excel(r"C:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\data\ESG_DATA_S&P500.xlsx")
dates = ['2013-04-01','2021-03-01']
start_year = dates[0]
end_year = dates[1]

Bounds1 = Bounds(-1,2) #How willing we are to go short and to invest in one particular stock
Wanted_return = 0.20 #Only used when using the wanted_return constraint
maximum_risk = 0.10 #Either used when using maximum risk constraint or cmle portfolio
sharpe_type = "No_extra_constraint"  # rename to constraint, can be either Wanted_return or Maximum_risk or No_extra_constraint
risk_free_rate = 0.01

In [3]:
esg_data = data.esg_score_weight(df,weights,min_esg_score,max_esg_score) #ESG_DATA filtering function
full_data = data.stock_monthly_close(esg_data,dates) #Gives us the data we need from the stocks with relevant ESG_scores
prices,esgdata = data.seperate_full_data(full_data)
pct_returns = data.pct_returns_from_prices(prices)

parameters = portfolio.efficient_frontier_solo(pct_returns,Bounds1, sharpe_type,start_year,end_year, Wanted_return, maximum_risk)


c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:40: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,
c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:113: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,


Min. Risk = 2.875% => Return: 1.060%  Sharpe Ratio = 0.37
Max. Sharpe Ratio = 0.68 => Return: 3.59%  Risk: 5.292%


In [4]:
Min. Risk = 9.959% => Return: 12.686%  Sharpe Ratio = 1.27
Max. Sharpe Ratio = 2.35 => Return: 43.09%  Risk: 18.354%
weights = efficient_frontier.weights_of_portfolio(prices,parameters)
weights

SyntaxError: invalid syntax (2538590959.py, line 1)